<a href="https://colab.research.google.com/github/LuminoWeiss/GitStart/blob/master/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/cs4246/gym-grid-driving.git
!cd gym-grid-driving
!pip install git+https://github.com/cs4246/gym-grid-driving.git


Cloning into 'gym-grid-driving'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 139 (delta 45), reused 135 (delta 43), pack-reused 0
Receiving objects: 100% (139/139), 27.08 KiB | 1.69 MiB/s, done.
Resolving deltas: 100% (45/45), done.
  Cloning https://github.com/cs4246/gym-grid-driving.git to /tmp/pip-req-build-77902pvx
  Running command git clone -q https://github.com/cs4246/gym-grid-driving.git /tmp/pip-req-build-77902pvx
  Created wheel for gym-grid-driving: filename=gym_grid_driving-0.0.1-cp36-none-any.whl size=8624 sha256=7a37e4e7cdb1ac503e27159f13bfd8b10199da0c2c4bb400aa63282bd466f7e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-a1q6fe1o/wheels/e1/30/f2/157c0938ab9bfe9c10c29c9fcab8392f587c9d141f215b67ca
Successfully built gym-grid-driving


In [0]:
try:
    from runner.abstracts import Agent
except:
    class Agent(object): pass
import random
import os
import torch
from models import *
from env import construct_task1_env, construct_task2_env
import collections
import numpy as np
import random
import math
import gym

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from gym_grid_driving.envs.grid_driving import GridDrivingEnv

from gym_grid_driving.envs.grid_driving import LaneSpec
import gym_grid_driving

import datetime

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
script_path = os.path.dirname(os.path.realpath('/content'))
model_path = '/content/model.pt'
print(model_path)


/content/model.pt


In [0]:
# Hyperparameters --- don't change, RL is very sensitive
learning_rate = 0.00001
gamma         = 0.98
buffer_limit  = 5000
batch_size    = 32
max_episodes  = 8000
t_max         = 600
min_buffer    = 1000
target_update = 20 # episode(s)
train_steps   = 10
max_epsilon   = 0.10
min_epsilon   = 0.01
epsilon_decay = 500
print_interval= 20

Transition = collections.namedtuple('Transition', ('state', 'action', 'reward', 'next_state', 'done'))

class DenseReward:
    FINISH_REWARD   = 100
    MISSED_REWARD   = -5
    CRASH_REWARD    = -20
    TIMESTEP_REWARD = -1

class AtariDQN(DQN):
    def construct(self):
        self.features = nn.Sequential(
            nn.Conv2d(self.input_shape[0], 32, kernel_size=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU()
        )
        self.layers = nn.Sequential(
            nn.Linear(self.feature_size(), 512),
            nn.ReLU(),
            nn.Linear(512, self.num_actions)
        )


    def act(self, state, epsilon=0.0):
        if not isinstance(state, torch.FloatTensor):
            state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        '''
        FILL ME : This function should return epsilon-greedy action.

        Input:
            * `state` (`torch.tensor` [batch_size, channel, height, width])
            * `epsilon` (`float`): the probability for epsilon-greedy

        Output: action (`Action` or `int`): representing the action to be taken.
                if action is of type `int`, it should be less than `self.num_actions`
        '''
        sample = random.random()
        if sample > epsilon:
            with torch.no_grad():
                state = autograd.Variable(state)
            q_value = self.forward(state)
            action = q_value.max(1)[1].item()
        else:
            action = random.randrange(self.num_actions)

        return action


class ReplayBuffer():
    def __init__(self, buffer_limit=buffer_limit):
        '''
        FILL ME : This function should initialize the replay buffer `self.buffer` with maximum size of `buffer_limit` (`int`).
                  len(self.buffer) should give the current size of the buffer `self.buffer`.
        '''
        self.buffer = []
        self.capacity = buffer_limit
        self.position = 0
    
    def push(self, transition):
        '''
        FILL ME : This function should store the transition of type `Transition` to the buffer `self.buffer`.

        Input:
            * `transition` (`Transition`): tuple of a single transition (state, action, reward, next_state, done).
                                           This function might also need to handle the case  when buffer is full.

        Output:
            * None
        '''
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)

        self.buffer[self.position] = Transition(*transition)
        self.position = (self.position + 1) % self.capacity


    
    def sample(self, batch_size):
        '''
        FILL ME : This function should return a set of transitions of size `batch_size` sampled from `self.buffer`

        Input:
            * `batch_size` (`int`): the size of the sample.

        Output:
            * A 5-tuple (`states`, `actions`, `rewards`, `next_states`, `dones`),
                * `states`      (`torch.tensor` [batch_size, channel, height, width])
                * `actions`     (`torch.tensor` [batch_size, 1])
                * `rewards`     (`torch.tensor` [batch_size, 1])
                * `next_states` (`torch.tensor` [batch_size, channel, height, width])
                * `dones`       (`torch.tensor` [batch_size, 1])
              All `torch.tensor` (except `actions`) should have a datatype `torch.float` and resides in torch device `device`.
        '''
        experiences = random.sample(self.buffer, batch_size)

        states = torch.from_numpy(np.stack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)

        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.stack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)


    def __len__(self):
        '''
        Return the length of the replay buffer.
        '''
        return len(self.buffer)



class ExampleAgent(Agent):
    '''
    An example agent that just output a random action.
    '''

    def __init__(self, *args, **kwargs):
        '''
        [OPTIONAL]
        Initialize the agent with the `test_case_id` (string), which might be important
        if your agent is test case dependent.

        For example, you might want to load the appropriate neural networks weight
        in this method.
        '''
        test_case_id = kwargs.get('test_case_id')
        self.model = get_model()


        '''
        # Uncomment to help debugging
        print('>>> __INIT__ >>>')
        print('test_case_id:', test_case_id)
        '''


    def initialize(self, **kwargs):
        '''
        [OPTIONAL]
        Initialize the agent.

        Input:
        * `env` (GridDrivingEnv): the environment of the test case
        * `fast_downward_path` (string): the path to the fast downward solver
        * `agent_speed_range` (tuple(float, float)): the range of speed of the agent
        * `gamma` (float): discount factor used for the task

        Output:
        * None

        This function will be called once before the evaluation.
        '''
        #env                 = kwargs.get('env') # WARNING: not available in the 2nd task
        fast_downward_path  = kwargs.get('fast_downward_path')
        agent_speed_range   = kwargs.get('agent_speed_range')
        gamma               = kwargs.get('gamma')

        # Uncomment to help debugging
        # print('>>> INITIALIZE >>>')
        # print('env:', env)
        # print('fast_downward_path:', fast_downward_path)
        # print('agent_speed_range:', agent_speed_range)
        # print('gamma:', gamma)


    def step(self, state, epsilon=0.0, *args, **kwargs):
        ''' 
        [REQUIRED]
        Step function of the agent which computes the mapping from state to action.
        As its name suggests, it will be called at every step.
        
        Input:
        * `state`: task 1 - `GridDrivingState`
                   task 2 -  tensor of dimension `[channel, height, width]`, with 
                            `channel=[cars, agent, finish_position, occupancy_trails]` for task 2

        Output:
        * `action`: `int` representing the index of an action or instance of class `Action`.
                    In this example, we only return a random action
        '''
        
        return self.model.act(state, epsilon)


        # Uncomment to help debugging
        '''
        print('>>> STEP >>>')
        print('state:', state)
        return model.act(state, epsilon)
        #return random.randrange(5)
        '''
        




    def update(self, *args, **kwargs):
        '''
        [OPTIONAL]
        Update function of the agent. This will be called every step after `env.step` is called.
        
        Input:
        * `state`: task 1 - `GridDrivingState`
                   task 2 -  tensor of dimension `[channel, height, width]`, with 
                            `channel=[cars, agent, finish_position, occupancy_trails]` for task 2
        * `action` (`int` or `Action`): the executed action (given by the agent through `step` function)
        * `reward` (float): the reward for the `state`
        * `next_state` (same type as `state`): the next state after applying `action` to the `state`
        * `done` (`int`): whether the `action` induce terminal state `next_state`
        * `info` (dict): additional information (can mostly be disregarded)

        Output:
        * None

        This function might be useful if you want to have policy that is dependant to its past.
        '''
        state       = kwargs.get('state')
        action      = kwargs.get('action')
        reward      = kwargs.get('reward')
        next_state  = kwargs.get('next_state')
        done        = kwargs.get('done')
        info        = kwargs.get('info')

        # Uncomment to help debugging
        # print('>>> UPDATE >>>')
        # print('state:', state)
        # print('action:', action)
        # print('reward:', reward)
        # print('next_state:', next_state)
        # print('done:', done)
        # print('info:', info)



def create_agent(test_case_id, *args, **kwargs):
    '''
    Method that will be called to create your agent during testing.
    You can, for example, initialize different class of agent depending on test case.
    '''
    return ExampleAgent(test_case_id=test_case_id)

def compute_loss(model, target, states, actions, rewards, next_states, dones):
    '''
    FILL ME : This function should compute the DQN loss function for a batch of experiences.

    Input:
        * `model`       : model network to optimize
        * `target`      : target network
        * `states`      (`torch.tensor` [batch_size, channel, height, width])
        * `actions`     (`torch.tensor` [batch_size, 1])
        * `rewards`     (`torch.tensor` [batch_size, 1])
        * `next_states` (`torch.tensor` [batch_size, channel, height, width])
        * `dones`       (`torch.tensor` [batch_size, 1])

    Output: scalar representing the loss.

    References:
        * MSE Loss  : https://pytorch.org/docs/stable/nn.html#torch.nn.MSELoss
        * Huber Loss: https://pytorch.org/docs/stable/nn.html#torch.nn.SmoothL1Loss
    '''
    # Get max predicted Q values (for next states) from target model
    Q_targets_next = target(next_states).detach().max(1)[0].unsqueeze(1)

    # Compute Q targets for current states
    Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

    # Get expected Q values from local model
    Q_expected = model(states).gather(1, actions)

    # Compute loss
    loss = F.mse_loss(Q_expected, Q_targets)
    return loss

def train(model_class, env):
    '''
    Train a model of instance `model_class` on environment `env` (`GridDrivingEnv`).
    
    It runs the model for `max_episodes` times to collect experiences (`Transition`)
    and store it in the `ReplayBuffer`. It collects an experience by selecting an action
    using the `model.act` function and apply it to the environment, through `env.step`.
    After every episode, it will train the model for `train_steps` times using the 
    `optimize` function.

    Output: `model`: the trained model.
    '''
    
    # Initialize model and target network
    model = model_class(env.observation_space.shape, env.action_space.n).to(device)
    target = model_class(env.observation_space.shape, env.action_space.n).to(device)
    target.load_state_dict(model.state_dict())
    target.eval()

    # Initialize replay buffer
    memory = ReplayBuffer()

    # Initialize rewards, losses, and optimizer
    rewards = []
    losses = []
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print (model.__class__.__name__)

    #for episode in range(3):
    for episode in range(max_episodes):
        epsilon = compute_epsilon(episode)
        state = env.reset()
        episode_rewards = 0.0
        
        #for t in range(10):
        for t in range(t_max):
            # Model takes action
            action = model.act(state, epsilon)
            # Apply the action to the environment
            next_state, reward, done, info = env._step(action)

            # Save transition to replay buffer
            memory.push(Transition(state, [action], [reward], next_state, [done]))

            state = next_state
            episode_rewards += reward
            if done:
                break
        rewards.append(episode_rewards)

        # Train the model if memory is sufficient
        if len(memory) > min_buffer:
            if np.mean(rewards[print_interval:]) < 0.1:
                print('Bad initialization. Please restart the training.')
                exit()
            for i in range(train_steps):
                loss = optimize(model, target, memory, optimizer)
                losses.append(loss.item())

        # Update target network every once in a while
        if episode % target_update == 0:
            target.load_state_dict(model.state_dict())

        if episode % print_interval == 0 and episode > 0:
            print("[Episode {}]\tavg rewards : {:.3f},\tavg loss: : {:.6f},\tbuffer size : {},\tepsilon : {:.1f}%".format(
                            episode, np.mean(rewards[print_interval:]), np.mean(losses[print_interval*10:]), len(memory), epsilon*100))
    return model

def optimize(model, target, memory, optimizer):
    '''
    Optimize the model for a sampled batch with a length of `batch_size`
    '''
    batch = memory.sample(batch_size)
    loss = compute_loss(model, target, *batch)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss

def compute_epsilon(episode):
    '''
    Compute epsilon used for epsilon-greedy exploration
    '''
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * math.exp(-1. * episode / epsilon_decay)
    return epsilon

def save_model(model):
    '''
    Save `model` to disk. Location is specified in `model_path`. 
    '''
    data = (model.__class__.__name__, model.state_dict(), model.input_shape, model.num_actions)
    torch.save(data, model_path)
    
def get_model():
    '''
    Load `model` from disk. Location is specified in `model_path`.
    '''
    model_class, model_state_dict, input_shape, num_actions = torch.load(model_path)
    model = eval(model_class)(input_shape, num_actions).to(device)
    model.load_state_dict(model_state_dict)
    return model


In [0]:
print(model_path)

/content/model.pt


In [0]:

def train2(model_class, env):
    '''
    Train a model of instance `model_class` on environment `env` (`GridDrivingEnv`).
    
    It runs the model for `max_episodes` times to collect experiences (`Transition`)
    and store it in the `ReplayBuffer`. It collects an experience by selecting an action
    using the `model.act` function and apply it to the environment, through `env.step`.
    After every episode, it will train the model for `train_steps` times using the 
    `optimize` function.

    Output: `model`: the trained model.
    '''
    
    # Initialize model and target network
    model = get_model()
    target = get_model()
    target.load_state_dict(model.state_dict())
    target.eval()

    # Initialize replay buffer
    memory = ReplayBuffer()

    # Initialize rewards, losses, and optimizer
    rewards = []
    losses = []
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print (model.__class__.__name__)

    #for episode in range(3):
    for episode in range(max_episodes):
        epsilon = compute_epsilon(episode)
        state = env.reset()
        episode_rewards = 0.0
        
        #for t in range(10):
        for t in range(t_max):
            # Model takes action
            action = model.act(state, epsilon)
            # Apply the action to the environment
            next_state, reward, done, info = env._step(action)

            # Save transition to replay buffer
            memory.push(Transition(state, [action], [reward], next_state, [done]))

            state = next_state
            episode_rewards += reward
            if done:
                break
        rewards.append(episode_rewards)

        # Train the model if memory is sufficient
        if len(memory) > min_buffer:
            if np.mean(rewards[print_interval:]) < 0.1:
                print('Bad initialization. Please restart the training.')
                exit()
            for i in range(train_steps):
                loss = optimize(model, target, memory, optimizer)
                losses.append(loss.item())

        # Update target network every once in a while
        if episode % target_update == 0:
            target.load_state_dict(model.state_dict())

        if episode % print_interval == 0 and episode > 0:
            print("[Episode {}]\tavg rewards : {:.3f},\tavg loss: : {:.6f},\tbuffer size : {},\tepsilon : {:.1f}%".format(
                            episode, np.mean(rewards[print_interval:]), np.mean(losses[print_interval*10:]), len(memory), epsilon*100))
    return model

In [0]:
def get_env():
    '''
    Get the sample test cases for training and testing.
    '''

    config = {'observation_type': 'tensor', 'agent_speed_range': [-3, -1], 'width': 50,
          'lanes': [LaneSpec(cars=3, speed_range=[-2, -1]),
                    LaneSpec(cars=2, speed_range=[-2, -1]),
                    LaneSpec(cars=3, speed_range=[-1, -1]),
                    LaneSpec(cars=2, speed_range=[-3, -1]),
                    LaneSpec(cars=3, speed_range=[-2, -1]),
                    LaneSpec(cars=2, speed_range=[-2, -1]),
                    LaneSpec(cars=3, speed_range=[-3, -2]),
                    LaneSpec(cars=2, speed_range=[-1, -1]),
                    LaneSpec(cars=3, speed_range=[-2, -1]),
                    LaneSpec(cars=2, speed_range=[-2, -2])]
        }
    return gym.make('GridDriving-v0', **config)


In [0]:

if __name__ == '__main__':
    import sys
    import time
    from env import construct_task1_env, construct_task2_env
    from gym_grid_driving.envs.grid_driving import GridDrivingEnv

    FAST_DOWNWARD_PATH = "/fast_downward/"

    def test(agent, env, runs=1000, t_max=100):
        rewards = []
        for run in range(runs):
            state = env.reset()
            agent_init = {'fast_downward_path': FAST_DOWNWARD_PATH, 'agent_speed_range': (-3,-1), 'gamma' : 1}
            agent.initialize(**agent_init)
            episode_rewards = 0.0
            for t in range(t_max):
                action = agent.step(state)
                next_state, reward, done, info = env.step(action)
                full_state = {
                    'state': state, 'action': action, 'reward': reward, 'next_state': next_state, 
                    'done': done, 'info': info
                }
                agent.update(**full_state)
                state = next_state
                episode_rewards += reward
                if done:
                    break
            rewards.append(episode_rewards)
        avg_rewards = sum(rewards)/len(rewards)
        print("{} run(s) avg rewards : {:.1f}".format(runs, avg_rewards))
        return avg_rewards

    def timed_test(task):
        start_time = time.time()
        rewards = []
        for tc in task['testcases']:
            agent = create_agent(tc['id']) # `test_case_id` is unique between the two task 
            print("[{}]".format(tc['id']), end=' ')
            avg_rewards = test(agent, tc['env'], tc['runs'], tc['t_max'])
            rewards.append(avg_rewards)
        point = sum(rewards)/len(rewards)
        elapsed_time = time.time() - start_time

        print('Point:', point)

        for t, remarks in [(0.4, 'fast'), (0.6, 'safe'), (0.8, 'dangerous'), (1.0, 'time limit exceeded')]:
            if elapsed_time < task['time_limit'] * t:
                print("Local runtime: {} seconds --- {}".format(elapsed_time, remarks))
                print("WARNING: do note that this might not reflect the runtime on the server.")
                break

    def get_task(task_id):
        if task_id == 1:
            test_case_id = 'task1_test'
            return { 
                'time_limit' : 600,
                'testcases' : [{'id' : test_case_id, 'env' : construct_task1_env(), 'runs' : 1, 't_max' : 50}]
                }
        elif task_id == 2:
            tcs = [('t2_tmax50', 50), ('t2_tmax40', 40)]
            return {
                'time_limit': 600,
                'testcases': [{ 'id': tc, 'env': construct_task2_env(), 'runs': 300, 't_max': t_max } for tc, t_max in tcs]
            }
        else:
            raise NotImplementedError

    try:
        task_id = 2
        task_type = 'train'
    except:
        print('Run agent on an example task.')
        print('Usage: python __init__.py <task number>')
        print('Example:\n   python __init__.py 2')
        exit()

    print('Testing on Task {}'.format(task_id))

    task = get_task(task_id)

    if task_type == "train":
        env = construct_task2_env()
        model = train2(AtariDQN,env)
        save_model(model)
    else:
        print ("get model")
        #model = get_model()
    print ("start testing............")
    timed_test(task)

Testing on Task 2
AtariDQN


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[Episode 20]	avg rewards : 10.000,	avg loss: : nan,	buffer size : 584,	epsilon : 9.6%
[Episode 40]	avg rewards : 3.333,	avg loss: : nan,	buffer size : 1067,	epsilon : 9.3%
[Episode 60]	avg rewards : 4.634,	avg loss: : 0.458321,	buffer size : 1654,	epsilon : 9.0%
[Episode 80]	avg rewards : 4.426,	avg loss: : 0.581849,	buffer size : 2149,	epsilon : 8.7%
[Episode 100]	avg rewards : 4.568,	avg loss: : 0.585220,	buffer size : 2648,	epsilon : 8.4%
[Episode 120]	avg rewards : 5.149,	avg loss: : 0.569508,	buffer size : 3229,	epsilon : 8.1%
[Episode 140]	avg rewards : 5.455,	avg loss: : 0.553370,	buffer size : 3829,	epsilon : 7.8%
[Episode 160]	avg rewards : 5.461,	avg loss: : 0.542332,	buffer size : 4383,	epsilon : 7.5%
[Episode 180]	avg rewards : 5.528,	avg loss: : 0.522944,	buffer size : 4973,	epsilon : 7.3%
[Episode 200]	avg rewards : 5.249,	avg loss: : 0.511156,	buffer size : 5000,	epsilon : 7.0%
[Episode 220]	avg rewards : 5.423,	avg loss: : 0.500156,	buffer size : 5000,	epsilon : 6.8%
[E

In [0]:
print(datetime.datetime.now())

2019-11-16 23:19:38.197427


In [0]:

if __name__ == '__main__':
    import sys
    import time
    from env import construct_task1_env, construct_task2_env
    from gym_grid_driving.envs.grid_driving import GridDrivingEnv

    FAST_DOWNWARD_PATH = "/fast_downward/"

    def test(agent, env, runs=1000, t_max=100):
        rewards = []
        for run in range(runs):
            state = env.reset()
            agent_init = {'fast_downward_path': FAST_DOWNWARD_PATH, 'agent_speed_range': (-3,-1), 'gamma' : 1}
            agent.initialize(**agent_init)
            episode_rewards = 0.0
            for t in range(t_max):
                action = agent.step(state)
                next_state, reward, done, info = env.step(action)
                full_state = {
                    'state': state, 'action': action, 'reward': reward, 'next_state': next_state, 
                    'done': done, 'info': info
                }
                agent.update(**full_state)
                state = next_state
                episode_rewards += reward
                if done:
                    break
            rewards.append(episode_rewards)
        avg_rewards = sum(rewards)/len(rewards)
        print("{} run(s) avg rewards : {:.1f}".format(runs, avg_rewards))
        return avg_rewards

    def timed_test(task):
        start_time = time.time()
        rewards = []
        for tc in task['testcases']:
            agent = create_agent(tc['id']) # `test_case_id` is unique between the two task 
            print("[{}]".format(tc['id']), end=' ')
            avg_rewards = test(agent, tc['env'], tc['runs'], tc['t_max'])
            rewards.append(avg_rewards)
        point = sum(rewards)/len(rewards)
        elapsed_time = time.time() - start_time

        print('Point:', point)

        for t, remarks in [(0.4, 'fast'), (0.6, 'safe'), (0.8, 'dangerous'), (1.0, 'time limit exceeded')]:
            if elapsed_time < task['time_limit'] * t:
                print("Local runtime: {} seconds --- {}".format(elapsed_time, remarks))
                print("WARNING: do note that this might not reflect the runtime on the server.")
                break

    def get_task(task_id):
        if task_id == 1:
            test_case_id = 'task1_test'
            return { 
                'time_limit' : 600,
                'testcases' : [{'id' : test_case_id, 'env' : construct_task1_env(), 'runs' : 1, 't_max' : 50}]
                }
        elif task_id == 2:
            tcs = [('t2_tmax50', 50), ('t2_tmax40', 40)]
            return {
                'time_limit': 600,
                'testcases': [{ 'id': tc, 'env': construct_task2_env(), 'runs': 300, 't_max': t_max } for tc, t_max in tcs]
            }
        else:
            raise NotImplementedError

    try:
        task_id = 2
        #task_type = 'train'
    except:
        print('Run agent on an example task.')
        print('Usage: python __init__.py <task number>')
        print('Example:\n   python __init__.py 2')
        exit()

    print('Testing on Task {}'.format(task_id))

    task = get_task(task_id)
    '''
    if task_type == "train":
        env = construct_task2_env()
        model = train2(AtariDQN,env)
        save_model(model)
    else:
        print ("get model")
    '''
    model = get_model()
    print ("start testing............")
    timed_test(task)

Testing on Task 2
start testing............
[t2_tmax50] 300 run(s) avg rewards : 8.1
[t2_tmax40] 300 run(s) avg rewards : 8.1
Point: 8.1
Local runtime: 282.1538383960724 seconds --- safe
